In [1]:
# script for analyzing sensitivity results

In [61]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics')
import kineticfun


import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load the base model
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model'
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/naive_rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)


In [4]:
# base_delay = 0.14857535  # /work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model -- original 24 hour model
base_delay = 0.006115998233733183  # /work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model -- naive 24 hour model



In [5]:
# load the csv to get the ignition delay times
# csv_file = '/scratch/harris.se/autoscience/local_sensitivity/practice.csv'
# csv_file = '/scratch/harris.se/autoscience/local_sensitivity/ignition_delays_007.csv'
csv_file = '/scratch/harris.se/autoscience/naive_model/sensitivity/results/ignition_delays_0007.csv'
df = pd.read_csv(csv_file, skiprows=1, names=['model', 'delay'])

In [70]:
# get the parameters that make the biggest difference
delays = df['delay'].values
for i in range(0, len(delays)):
    if delays[i] <= 0:
        delays[i] = base_delay

        
order = np.arange(0, len(delays))
differences = np.abs(delays - base_delay)
most_sensitive = [x for _, x in sorted(zip(differences, order))][::-1]

In [71]:
most_sensitive[1]

108

In [72]:
# print the top parameters


# lookup up manually:
# 87 -> 313
# 77 -> 288
# 34 -> 247

for i in range(0, 500):
    index = most_sensitive[i]
    if index > len(reaction_list):
        index = index - len(reaction_list)
        print(index, '\t', species_list[index])
    else:
        try:
            print(index, '\t', reaction_list[index], '\t', reaction_list[index].family)
        except AttributeError:
            print(index, '\t', reaction_list[index], '\t', 'PDEP')

87 	 O2(2) + [CH2]CCC(21) <=> [O]O(22) + C=CCC(46) 	 Disproportionation
108 	 CC(CCOO)O[O](123) <=> [OH](33) + CC(CC=O)OO(119) 	 intra_H_migration
55 	 C[CH]CC(19) + CCC(C)OO(59) <=> CCC(C)O[O](57) + butane(1) 	 H_Abstraction
77 	 [OH](33) + butane(1) <=> H2O(8) + [CH2]CCC(21) 	 H_Abstraction
37 	 O2(2) + C[CH]CC(19) <=> [O]O(22) + CC=CC(54) 	 Disproportionation
34 	 [OH](33) + butane(1) <=> H2O(8) + C[CH]CC(19) 	 H_Abstraction
54 	 CC(CCO[O])OO(104) <=> [OH](33) + CC(=O)CCOO(121) 	 intra_H_migration
123 	 C[CH]CC(19) + CC(CCOO)OO(124) <=> CC(CCOO)O[O](123) + butane(1) 	 H_Abstraction
306 	 [O]O(22) + [O]O(22) <=> O2(2) + OO(38) 	 H_Abstraction
24 	 CCOO(41) + C[CH]CC(19) <=> CCO[O](27) + butane(1) 	 H_Abstraction
485 	 [OH](33) + CH2O(9) <=> H2O(8) + [CH]=O(177) 	 H_Abstraction
410 	 [OH](33) + CC=O(34) <=> H2O(8) + C=C[O](212) 	 H_Abstraction
74 	 [CH2]CCC(21) + CCC(C)OO(59) <=> CCC(C)O[O](57) + butane(1) 	 H_Abstraction
305 	 [O]O(22) + C[CH2](16) <=> OO(38) + C2H4(11) 	 Disproporti

In [73]:
# convert the reaction numbers to primary model numbers for AutoTST calculation
for i in range(0, 50):
    rxn_smiles = kineticfun.reaction2smiles(reaction_list[most_sensitive[i]])
    primary_model_index = kineticfun.reaction_smiles2index(rxn_smiles)
    print(most_sensitive[i], '\t', primary_model_index, reaction_list[most_sensitive[i]], )


87 	 313 O2(2) + [CH2]CCC(21) <=> [O]O(22) + C=CCC(46)
108 	 -1 CC(CCOO)O[O](123) <=> [OH](33) + CC(CC=O)OO(119)
55 	 1701 C[CH]CC(19) + CCC(C)OO(59) <=> CCC(C)O[O](57) + butane(1)
77 	 288 [OH](33) + butane(1) <=> H2O(8) + [CH2]CCC(21)
37 	 278 O2(2) + C[CH]CC(19) <=> [O]O(22) + CC=CC(54)
34 	 247 [OH](33) + butane(1) <=> H2O(8) + C[CH]CC(19)
54 	 -1 CC(CCO[O])OO(104) <=> [OH](33) + CC(=O)CCOO(121)
123 	 808 C[CH]CC(19) + CC(CCOO)OO(124) <=> CC(CCOO)O[O](123) + butane(1)
306 	 18 [O]O(22) + [O]O(22) <=> O2(2) + OO(38)
24 	 1646 CCOO(41) + C[CH]CC(19) <=> CCO[O](27) + butane(1)
485 	 127 [OH](33) + CH2O(9) <=> H2O(8) + [CH]=O(177)
410 	 -1 [OH](33) + CC=O(34) <=> H2O(8) + C=C[O](212)
74 	 1700 [CH2]CCC(21) + CCC(C)OO(59) <=> CCC(C)O[O](57) + butane(1)
305 	 213 [O]O(22) + C[CH2](16) <=> OO(38) + C2H4(11)
326 	 1085 COO(91) + C[CH]CC(19) <=> CO[O](28) + butane(1)
420 	 -1 C[CH]CC(19) + OOCCCCOO(1883) <=> [O]OCCCCOO(229) + butane(1)
30 	 281 O2(2) + [CH2]CC(C)OO(72) <=> CC(CCO[O])OO(104)

In [74]:
for i in range(0, len(delays)):
    if most_sensitive[i] > len(reaction_list):
        sp_index = most_sensitive[i] - len(reaction_list)
        print(sp_index, '\t', species_list[sp_index])

100 	 C=CC(C)CC=CC(8935)
99 	 C=CC(C)O[O](8788)
98 	 CC=CCO[O](8787)
97 	 CC1OC=CO1(6802)
96 	 CC1OC1C=O(6772)
95 	 [C]=C-2(5810)
94 	 O=C(O)[CH]O(4710)
93 	 O=CC(=O)O(3420)
92 	 [O]CC(=O)O(3414)
91 	 O=C(O)COO(3121)
90 	 [O]OCC(=O)O(3083)
89 	 C1CO1(2791)
88 	 C#C(2633)
87 	 C=CC=O(2578)
86 	 [CH2]C(=O)O(2146)
85 	 OOCCCCOO(1883)
84 	 [O]CCCCOO(1870)
83 	 CO-2(969)
82 	 C=CC(C)[O](958)
81 	 OCCC1CO1(837)
80 	 [CH2]C(CCO)OO(828)
79 	 [O]OCC=O(741)
78 	 [O]CCC=O(716)
77 	 OO[CH]CCOO(307)
76 	 [O]OCC(=O)COO(293)
75 	 O=C1COC1(287)
74 	 C=C([O])C[O](280)
73 	 C=C([O])COO(272)
72 	 CC(=O)CO[O](262)
71 	 C=C=O(249)
70 	 [O]OCCCCOO(229)
69 	 C=CO(218)
68 	 CC(C=C[O])OO(215)
67 	 C=C[O](212)
66 	 CC([O])CC=O(209)
65 	 O=CCCOO(200)
64 	 [CH]=O(177)
63 	 [CH2]CO(174)
62 	 [CH2]CCCOO(169)
61 	 C[CH]CCOO(168)
60 	 CCCCO[O](161)
59 	 C[C]=O(148)
58 	 C=C(C)[O](147)
57 	 CC(=O)CC[O](144)
56 	 C[C]CC-2(130)
55 	 [CH2]C=CC(128)
54 	 CC(CCOO)OO(124)
53 	 CC(CCOO)O[O](123)
52 	 CC(=O)CCOO(121)
51 	 CC(

In [ ]:
len(reaction_list) + len(species_list)